In [1]:
import numpy as np
import pandas as pd

In [2]:
match = pd.read_csv('ODI_Match_Info.csv')
delivery = pd.read_csv('ODI_Match_Data.csv', low_memory=False)

In [3]:
match.info

<bound method DataFrame.info of            id   season          city        date        team1        team2  \
0     1389389  2023/24        Indore  2023/09/24        India    Australia   
1     1336129     2023    Nottingham  2023/09/23      England      Ireland   
2     1395701     2023         Dhaka  2023/09/23  New Zealand   Bangladesh   
3     1389388  2023/24    Chandigarh  2023/09/22    Australia        India   
4     1395700     2023         Dhaka  2023/09/21  New Zealand   Bangladesh   
...       ...      ...           ...         ...          ...          ...   
2374    65634  2002/03        Sydney  2003/01/09    Sri Lanka    Australia   
2375    64817  2002/03    Wellington  2003/01/08  New Zealand        India   
2376    64816  2002/03    Queenstown  2003/01/04        India  New Zealand   
2377    64815  2002/03  Christchurch  2003/01/01        India  New Zealand   
2378    64814  2002/03        Napier  2002/12/29  New Zealand        India   

      toss_winner toss_decision

In [4]:
delivery.info

<bound method DataFrame.info of          match_id   season  start_date                           venue  \
0         1389389  2023/24  2023-09-24  Holkar Cricket Stadium, Indore   
1         1389389  2023/24  2023-09-24  Holkar Cricket Stadium, Indore   
2         1389389  2023/24  2023-09-24  Holkar Cricket Stadium, Indore   
3         1389389  2023/24  2023-09-24  Holkar Cricket Stadium, Indore   
4         1389389  2023/24  2023-09-24  Holkar Cricket Stadium, Indore   
...           ...      ...         ...                             ...   
1265098     64814  2002/03  2002-12-29             McLean Park, Napier   
1265099     64814  2002/03  2002-12-29             McLean Park, Napier   
1265100     64814  2002/03  2002-12-29             McLean Park, Napier   
1265101     64814  2002/03  2002-12-29             McLean Park, Napier   
1265102     64814  2002/03  2002-12-29             McLean Park, Napier   

         innings  ball batting_team bowling_team     striker   non_striker  \
0

In [5]:
match.shape

(2379, 18)

In [6]:
delivery.shape

(1265103, 23)

In [7]:
print(delivery.columns)

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'cricsheet_id'],
      dtype='object')


In [8]:
delivery['total_runs'] = delivery['runs_off_bat'] + delivery['extras'].fillna(0)


In [9]:
finalscoredf = delivery.groupby(['match_id', 'innings']).sum()['total_runs'].reset_index()


In [10]:
finalscoredf

,match_id,innings,total_runs
0,64814,1,254
1,64814,2,219
2,64815,1,108
3,64815,2,109
4,64816,1,122
...,...,...,...
4696,1390346,1,268
4697,1390346,2,209
4698,1395700,1,136
4699,1395701,1,254


In [11]:
finalscoredf =finalscoredf[finalscoredf['innings']==1]

In [12]:
matchdf = match.merge(finalscoredf[['match_id', 'total_runs']], left_on='id', right_on='match_id')

In [13]:
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'    
]

In [14]:
matchdf = matchdf[matchdf['team1'].isin(teams)]
matchdf = matchdf[matchdf['team2'].isin(teams)]

In [15]:
matchdf = matchdf[matchdf['dl_applied']==0]

In [16]:
matchdf = matchdf[['match_id','city','winner','total_runs']]

In [17]:
deliverydf = matchdf.merge(delivery,on='match_id')

In [18]:
deliverydf = deliverydf[deliverydf['innings'] == 2]

In [19]:
pd.set_option('display.max_columns', None)
deliverydf=deliverydf.dropna(subset=['city','winner'])

In [20]:
print(deliverydf['total_runs_y'].dtype)
deliverydf['currentscore'] = deliverydf.groupby('match_id')['total_runs_y'].cumsum()

int64


In [21]:
deliverydf['runs_left'] = deliverydf['total_runs_x'] - deliverydf['currentscore'] + 1
deliverydf

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,total_runs_y,currentscore,runs_left
303,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.1,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255
304,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.2,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255
305,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.3,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255
306,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.4,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,1,1,254
307,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.5,Bangladesh,New Zealand,Liton Das,Tamim Iqbal,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,1,254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789641,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,0,216,39
789642,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,KD Mills,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,1,217,38
789643,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,J Srinath,NaN,NaN,64814,0,217,38
789644,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,KD Mills,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,2,219,36


In [22]:
deliverydf['over'] = deliverydf['ball'].apply(lambda x:str(x).split(".")[0])
deliverydf['ballofover'] = deliverydf['ball'].apply(lambda x:str(x).split(".")[1])
deliverydf



,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,total_runs_y,currentscore,runs_left,over,ballofover
303,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.1,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,1
304,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.2,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,2
305,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.3,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,3
306,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.4,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,1,1,254,0,4
307,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.5,Bangladesh,New Zealand,Liton Das,Tamim Iqbal,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,1,254,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789641,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,0,216,39,42,6
789642,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,KD Mills,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,1,217,38,43,1
789643,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,J Srinath,NaN,NaN,64814,0,217,38,43,2
789644,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,KD Mills,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,2,219,36,43,3


In [23]:
deliverydf['over'] = pd.to_numeric(deliverydf['over'], errors='coerce')
deliverydf['ballofover'] = pd.to_numeric(deliverydf['ballofover'], errors='coerce')
deliverydf['balls_left'] = 300 - (deliverydf['over'] * 6 + deliverydf['ballofover'])

deliverydf

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,total_runs_y,currentscore,runs_left,over,ballofover,balls_left
303,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.1,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,1,299
304,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.2,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,2,298
305,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.3,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,3,297
306,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.4,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,1,1,254,0,4,296
307,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.5,Bangladesh,New Zealand,Liton Das,Tamim Iqbal,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,1,254,0,5,295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789641,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,0,216,39,42,6,42
789642,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,KD Mills,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,1,217,38,43,1,41
789643,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,J Srinath,NaN,NaN,64814,0,217,38,43,2,40
789644,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,KD Mills,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,2,219,36,43,3,39


In [24]:
deliverydf['player_dismissed'] = deliverydf['player_dismissed'].fillna("0")

In [25]:
deliverydf['player_dismissed'] = deliverydf['player_dismissed'].apply(lambda x:x if x == "0" else "1")

In [26]:
deliverydf['player_dismissed'] = deliverydf['player_dismissed'].astype('int')
deliverydf

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,total_runs_y,currentscore,runs_left,over,ballofover,balls_left
303,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.1,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1395701,0,0,255,0,1,299
304,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.2,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1395701,0,0,255,0,2,298
305,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.3,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1395701,0,0,255,0,3,297
306,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.4,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,1,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1395701,1,1,254,0,4,296
307,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.5,Bangladesh,New Zealand,Liton Das,Tamim Iqbal,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1395701,0,1,254,0,5,295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789641,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,0,216,39,42,6,42
789642,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,KD Mills,1,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,1,217,38,43,1,41
789643,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,1,NaN,NaN,64814,0,217,38,43,2,40
789644,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,KD Mills,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,2,219,36,43,3,39


In [27]:
wickets = deliverydf.groupby('match_id')['player_dismissed'].cumsum()
deliverydf['wickets_left'] = 10 - wickets
deliverydf.tail(6)

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,total_runs_y,currentscore,runs_left,over,ballofover,balls_left,wickets_left
789640,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.5,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,0,216,39,42,5,43,2
789641,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,0,216,39,42,6,42,2
789642,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,KD Mills,1,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,1,217,38,43,1,41,2
789643,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,1,NaN,NaN,64814,0,217,38,43,2,40,1
789644,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,KD Mills,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,2,219,36,43,3,39,1
789645,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.4,India,New Zealand,Z Khan,A Nehra,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,1,NaN,NaN,64814,0,219,36,43,4,38,0


In [28]:
deliverydf['crr'] = (deliverydf['currentscore']*6)/(300 - deliverydf['balls_left'])

In [29]:
deliverydf['rrr'] = (deliverydf['runs_left']*6)/deliverydf['balls_left']

In [30]:
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

deliverydf['result'] = deliverydf.apply(result,axis=1)

In [31]:
finaldf = deliverydf[['batting_team','bowling_team','city','runs_left','balls_left','wickets_left','total_runs_x','crr','rrr','result']]

In [32]:
finaldf = finaldf.sample(finaldf.shape[0])

finaldf

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,crr,rrr,result
314374,Bangladesh,West Indies,St Kitts,138,35,4,338,4.550943,23.657143,0
395658,England,Australia,Chester-le-Street,201,293,10,200,0.000000,4.116041,1
318751,England,Sri Lanka,Manchester,11,238,10,67,5.516129,0.277311,1
117486,Bangladesh,West Indies,Dublin,146,179,9,247,5.057851,4.893855,1
571359,England,New Zealand,London,227,247,10,266,4.528302,5.514170,0
...,...,...,...,...,...,...,...,...,...,...
323729,England,West Indies,Antigua,62,126,4,159,3.379310,2.952381,1
597440,Pakistan,India,Kanpur,223,241,8,294,7.322034,5.551867,0
622090,Sri Lanka,New Zealand,Grenada,140,221,9,219,6.075949,3.800905,1
302425,South Africa,Australia,Perth,301,296,10,300,0.000000,6.101351,0


In [33]:
finaldf.dropna(inplace=True)
finaldf=finaldf[finaldf['balls_left']!=0]
finaldf.sample()

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,crr,rrr,result
73275,Australia,India,Canberra,209,189,8,302,5.081081,6.634921,0


In [34]:
finaldf.columns

Index(['batting_team', 'bowling_team', 'city', 'runs_left', 'balls_left',
       'wickets_left', 'total_runs_x', 'crr', 'rrr', 'result'],
      dtype='object')

In [35]:
finaldf.to_csv('Win_Predictor_Data.csv')

In [36]:
X = finaldf.drop(['wickets_left', 'total_runs_x'], axis=1)
y = finaldf.iloc[:, -1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [37]:
X_train.describe

<bound method NDFrame.describe of         batting_team  bowling_team          city  runs_left  balls_left  \
386096    Bangladesh   West Indies        Khulna         24          85   
563814       England  South Africa    Nottingham         39         249   
379898       England         India         Kochi        151         100   
439500         India   West Indies       Antigua        209         267   
238597  South Africa   West Indies      Barbados        188         127   
...              ...           ...           ...        ...         ...   
404734      Pakistan     Sri Lanka        Mirpur         84         141   
247396      Pakistan   New Zealand    Wellington        243         230   
580166    Bangladesh      Pakistan        Lahore        151         124   
778282  South Africa    Bangladesh  Bloemfontein        108         297   
193310   West Indies   Afghanistan      St Lucia        172         210   

             crr       rrr  result  
386096  4.911628  1.694118  

In [38]:
y_train.describe

<bound method NDFrame.describe of 386096    1
563814    1
379898    0
439500    1
238597    0
         ..
404734    1
247396    0
580166    0
778282    1
193310    0
Name: result, Length: 249945, dtype: int64>

In [39]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier


trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse_output=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [41]:

# Different models for classification
models = {
   #'LogisticRegression': LogisticRegression(solver='newton-cg', random_state=1),
    'RandomForestClassifier': RandomForestClassifier(n_estimators=100, random_state=1),
    #'XGBClassifier': XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=1)
}

# Dictionary to store results
results = {}

# Iterate through each model and fit the data
for name, model in models.items():
    # Create a pipeline
    pipe = Pipeline(steps=[
        ('step1', trf),
        ('step2', model)
    ])

In [42]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', RandomForestClassifier(random_state=1))])

In [43]:
y_pred = pipe.predict(X_test)

In [44]:

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
    
    # Store the result
results[name] = {
        'Accuracy': accuracy
}

results

{'RandomForestClassifier': {'Accuracy': 1.0}}

In [45]:
X.columns

Index(['batting_team', 'bowling_team', 'city', 'runs_left', 'balls_left',
       'crr', 'rrr', 'result'],
      dtype='object')

In [46]:
X['city'].unique()

array(['St Kitts', 'Chester-le-Street', 'Manchester', 'Dublin', 'London',
       'Christchurch', 'Barbados', 'Chennai', 'Pune', 'North Sound',
       'Colombo', 'Cape Town', 'Karachi', 'Vadodara', 'Lahore', 'Napier',
       'Perth', 'Whangarei', 'Hamilton', 'Abu Dhabi', 'Trinidad',
       'Dunedin', 'Paarl', 'Wellington', 'Mirpur', 'Johannesburg',
       'Southampton', 'Auckland', 'Providence', 'Port Elizabeth', 'Leeds',
       'Centurion', 'Antigua', 'Rawalpindi', 'Guyana', 'Mumbai', 'Ranchi',
       'Nelson', 'Nottingham', 'Durban', 'Kanpur', 'Chandigarh',
       'Gwalior', 'Hambantota', 'Nagpur', 'Kochi', 'Chattogram',
       'Belfast', 'Bloemfontein', 'Delhi', 'Hobart', 'Hyderabad',
       'Gqeberha', 'Sydney', 'Cuttack', 'Lucknow', 'Birmingham',
       'Bridgetown', 'Dhaka', 'Jamaica', 'Dominica', 'Bogra', 'Ahmedabad',
       'Mount Maunganui', 'Brisbane', 'St Lucia', "St George's",
       'Adelaide', 'Jaipur', 'Dambulla', 'Kimberley', 'Grenada',
       'Tarouba', 'Bangalore', 'Bu